In [2]:
!pip install -Uqq fastbook nbdev ipywidgets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 246.1/246.1 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import fastbook
fastbook.setup_book()
from fastbook import *
from fastai.vision.widgets import *

from google.colab import drive
drive.mount('/content/gdrive')
BASE_DIR = '/content/gdrive/MyDrive/Colab Notebooks'

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

path = untar_data(URLs.MNIST_SAMPLE, data=f'{BASE_DIR}/data')

Mounted at /content/gdrive
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
def filesToStackedTensors(d1, d2):
  t = [tensor(Image.open(f)).to(device) for f in (path/d1/d2).ls().sorted()]
  return torch.stack(t).float()/255

def x_y(stacked_x, stacked_y):
  x = torch.cat([stacked_x, stacked_y]).view(-1, 28*28)
  y = tensor([1]*len(stacked_x) + [0]*len(stacked_y)).unsqueeze(1).to(device)
  return (x, y)

train_x, train_y = x_y(filesToStackedTensors('train', '3'), \
                       filesToStackedTensors('train', '7'))
valid_x, valid_y = x_y(filesToStackedTensors('valid', '3'), \
                       filesToStackedTensors('valid', '7'))

dset = list(zip(train_x, train_y))
valid_dset = list(zip(valid_x, valid_y))
dl = DataLoader(dset, batch_size=255, shuffle=True)
valid_dl = DataLoader(valid_dset, batch_size=255, shuffle=True)
dls = DataLoaders(dl, valid_dl)

In [11]:
# model

def linear1(x, w, b): return x@w + b

# model parameters

def init_params(size): return (torch.randn(size)).to(device).requires_grad_()

# loss

def loss(preds, y):
  s = preds.sigmoid()
  return torch.where((y == 1), 1-s, s).float().mean()

# optimiser

class BasicOptimiser:
  def __init__(self, params, lr):
    self.params = params
    self.lr = lr

  def step(self):
    for p in self.params:
      p.data = p.data - p.grad.data * self.lr

  def zero_grad(self):
    for p in self.params:
      p.grad = None

# metric

def batch_accuracy(xb, yb):
  p = xb.sigmoid()
  return ((p > 0.5).float() == yb).float().mean()

def batch_accuracy_dl(dl, model, w, b):
  accs = [batch_accuracy(model(xb, w, b), yb) for xb, yb in dl]
  return torch.stack(accs).mean()

# fit

def calc_gradient(xb, yb, model, w, b):
    p = model(xb, w, b)
    l = loss(p, yb)
    l.backward()

def train_epoch(dl, model, opt, w, b):
  for xb, yb in dl:
    calc_gradient(xb, yb, model, w, b)
    opt.step()
    opt.zero_grad()

def train_model(dl, model, opt, n, w, b, pr=False):
  for i in range(n):
    train_epoch(dl, model, opt, w, b)
    if pr:
      print(f'acc after {i} epochs: {batch_accuracy_dl(dl, model, w, b)}')

# putting it all together

weights = init_params((28*28, 1))
bias = init_params(1)
lr = 0.1
opt = BasicOptimiser((weights, bias), lr)
train_model(dl, linear1, opt, 10, weights, bias, pr=True)

# learner

acc after 0 epochs: 0.5407854914665222
acc after 1 epochs: 0.6012374758720398
acc after 2 epochs: 0.6659910082817078
acc after 3 epochs: 0.7181119322776794
acc after 4 epochs: 0.7670406699180603
acc after 5 epochs: 0.8082325458526611
acc after 6 epochs: 0.8354649543762207
acc after 7 epochs: 0.8567380905151367
acc after 8 epochs: 0.8685474991798401
acc after 9 epochs: 0.878666877746582


In [37]:
# model

# def model(xs, ys):
#   def init_params(size): return (torch.randn(size)).to(device).requires_grad_()
#   w = init_params(xs)
#   b = init_params(ys)
#   def linear1(x): return x@w + b
#   return (linear1, (w, b))

class Linear:
  def init_params(size):
    g = torch.Generator()
    g.manual_seed(5)
    return (torch.randn(size, generator=g)).to(device).requires_grad_()
  def __init__(self, xsz, ysz):
    w = init_params(xsz)
    b = init_params(ysz)
    self.parameters = [w, b]
  def model(self):
    return lambda xb: xb@self.parameters[0] + self.parameters[1]
  def params(self):
    return self.parameters

def batch_accuracy_dl(dl, model):
  accs = [batch_accuracy(model(xb), yb) for xb, yb in dl]
  return torch.stack(accs).mean()

# fit

def calc_gradient(xb, yb, model):
    p = model(xb)
    l = loss(p, yb)
    print(f'loss: {l}')
    l.backward()

def train_epoch(dl, model, opt):
  for xb, yb in dl:
    p = model(xb)
    l = loss(p, yb)
    # print(f'loss: {l}')
    l.backward()
    opt.step()
    opt.zero_grad()

def train_model(dl, model, opt, n, pr=False):
  for i in range(n):
    train_epoch(dl, model, opt)
    if pr:
      print(f'acc after {i} epochs: {batch_accuracy_dl(dl, model)}')

# putting it all together

lr = 1.
linear1 = Linear((28*28, 1), 1)
params = linear1.params()
opt = BasicOptimiser(params, lr)
train_model(dl, linear1.model(), opt, 10, pr=True)

acc after 0 epochs: 0.8831855654716492
acc after 1 epochs: 0.9352772235870361
acc after 2 epochs: 0.9481916427612305
acc after 3 epochs: 0.9562594294548035
acc after 4 epochs: 0.9611567258834839
acc after 5 epochs: 0.9646704792976379
acc after 6 epochs: 0.9665327072143555
acc after 7 epochs: 0.9683442115783691
acc after 8 epochs: 0.9700464606285095
acc after 9 epochs: 0.9712761640548706


In [22]:
for x in (1,2,3):
  print(x)

1
2
3
